In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
from pathlib import Path
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [51]:
#converting the png folders to jpg
from PIL import Image
os.chdir('..')
png_folder = 'Wheat__Yellow_Rust'
os.chdir(png_folder)

for file in glob.glob('*png'):
    img = Image.open(file).convert('RGB')
    jpg_file = file.replace('.png', '.jpg')
    img.save(jpg_file, 'JPEG')

In [77]:
# Change the working directory to the class folder
os.chdir('..')
os.chdir('Potato__Healthy(no_aug)')

# Ensure that the 'train', 'valid', and 'test' directories exist
if not os.path.isdir('train'):
    os.makedirs('train')
    os.makedirs('valid')
    os.makedirs('test')

# Move images to the 'train', 'valid', and 'test' folders
for c in random.sample(glob.glob('*.jpg'), 306):
    shutil.move(c, 'train')

for c in random.sample(glob.glob('*.jpg'), 50):
    shutil.move(c, 'valid')

for c in random.sample(glob.glob('*.jpg'), 50):
    shutil.move(c, 'test')

In [10]:
# Uniforming the size of the images
from PIL import Image, ImageEnhance

base_input = r"C:\Users\HP\Code\Plant_Disease\training\Plant_Disease_Dataset\PLD_wo_aug_256"
base_output = r"C:\Users\HP\Code\Plant_Disease\training\Plant_Disease_Dataset\PLD_wo_aug_224"
os.makedirs(base_output, exist_ok=True)

target_size = (224, 224)

# Loop through each class folder
for class_folder in os.listdir(base_input):
    class_path = os.path.join(base_input, class_folder)

    if os.path.isdir(class_path):
        # Create the class folder in the output directory
        output_class_path = os.path.join(base_output, class_folder)
        os.makedirs(output_class_path, exist_ok=True)

        # Loop through the subfolders
        for subset in [
            'Background_without_leaves', 
            'Corn__Gray_Leaf_Spot', 
            'Corn__Healthy',
            'Corn__Northern_Leaf_Blight',
            'Grape__Downey_Mildew',
            'Grape__Healthy',
            'Grape__Powdery_Mildew',
            'Olive__Healthy',
            'Olive__Peacock_Spot',
            'Olive__Rust_Mite',
            'Potato__Early_Blight',
            'Potato__Healthy',
            'Potato__Late_blight',
            'Tomato__Early_blight',
            'Tomato__Healthy',
            'Tomato__Late_Blight',
            'Wheat__Healthy',
            'Wheat__Septoria',
            'Wheat__Yellow_Rust']:
            subset_path = os.path.join(class_path, subset)

            if os.path.isdir(subset_path):
                output_subset_path = os.path.join(output_class_path, subset)
                os.makedirs(output_subset_path, exist_ok=True)

                # Process all .jpg and .png images in the current subset folder
                for ext in ('*.jpg'):
                    for file in glob.glob(os.path.join(subset_path, ext)):
                        try:
                            img = Image.open(file).convert('RGB')
                            img_resized = img.resize(target_size)

                            filename = os.path.basename(file)
                            img_resized.save(os.path.join(output_subset_path, filename))

                            
                        except Exception as e:
                            print(f"Failed to process {file}: {e}")


Failed to process C:\Users\HP\Code\Plant_Disease\training\Plant_Disease_Dataset\PLD_wo_aug_256\test\Background_without_leaves\.: [Errno 13] Permission denied: 'C:\\Users\\HP\\Code\\Plant_Disease\\training\\Plant_Disease_Dataset\\PLD_wo_aug_256\\test\\Background_without_leaves\\.'
Failed to process C:\Users\HP\Code\Plant_Disease\training\Plant_Disease_Dataset\PLD_wo_aug_256\test\Corn__Gray_Leaf_Spot\.: [Errno 13] Permission denied: 'C:\\Users\\HP\\Code\\Plant_Disease\\training\\Plant_Disease_Dataset\\PLD_wo_aug_256\\test\\Corn__Gray_Leaf_Spot\\.'
Failed to process C:\Users\HP\Code\Plant_Disease\training\Plant_Disease_Dataset\PLD_wo_aug_256\test\Corn__Healthy\.: [Errno 13] Permission denied: 'C:\\Users\\HP\\Code\\Plant_Disease\\training\\Plant_Disease_Dataset\\PLD_wo_aug_256\\test\\Corn__Healthy\\.'
Failed to process C:\Users\HP\Code\Plant_Disease\training\Plant_Disease_Dataset\PLD_wo_aug_256\test\Corn__Northern_Leaf_Blight\.: [Errno 13] Permission denied: 'C:\\Users\\HP\\Code\\Plant_Dis

In [2]:
# Data augumentation for less populated folders

def augment_image(img):
    # random horizontal flip
    if random.random() > 0.5:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)

    # random rotation
    angle = random.randint(-30, 30) 
    img = img.rotate(angle)

    # random zoom
    if random.random() > 0.5:
        width, height = img.size
        new_width = random.randint(int(width*0.8), int(width * 1.2))
        new_height = int(height * (new_width / width))
        img = img.resize((new_width, new_height))
        img = img.crop((0, 0, width, height))

    return img

input_folder = r"C:\Users\HP\Code\Plant_Disease\training\Plant_Disease_Dataset\PLD_wo_aug\Potato__Healthy(no_aug)"
output_folder = r"C:\Users\HP\Code\Plant_Disease\training\Plant_Disease_Dataset\PLD_wo_aug\Potato__Healthy(aug)"
os.makedirs(output_folder, exist_ok=True)
for file in glob.glob(os.path.join(input_folder, '*.jpg')):  
    img = Image.open(file).convert('RGB')

    for i in range(8):
        augmented_img = augment_image(img)
        augmented_img.save(os.path.join(output_folder, f"aug2_{os.path.basename(file)}"))



In [12]:
base_dir = Path("C:/Users/HP/Code/Plant_Disease/training/Plant_Disease_Dataset/PLD_wo_aug_256")
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3

In [13]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Plant_Disease_Dataset/PLD_wo_aug_256",
    shuffle=True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 9500 files belonging to 3 classes.


In [14]:
# splitting Data into train, val, test

train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir / "train",
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    shuffle = True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir / "valid",
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    shuffle = False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir / "test",
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    shuffle = False
)

Found 7600 files belonging to 19 classes.
Found 950 files belonging to 19 classes.
Found 950 files belonging to 19 classes.


In [15]:
# Optimizing

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [16]:
# layer, takes care of resizing the images the model will take as an input

resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])

In [17]:
# data augmentation layer

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
])

In [18]:
# Creating base model from the pre-trained model MNV2
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

In [19]:
image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)

In [20]:
# Freezing the convolutional base

base_model.trainable = False

In [21]:
# converting the features to a single element vector per image

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [22]:
# prediction layer

prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 1)


In [23]:
inputs = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = data_augmentation(inputs)
x = resize_and_rescale(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [151]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_4 (Sequential)            │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_2 (Sequential)            │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 8, 8, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │           1,281 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [24]:
base_learning_rate = 0.0001
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')])

In [25]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(val_ds)

30/30 ━━━━━━━━━━━━━━━━━━━━ 18s 458ms/step - accuracy: 0.0485 - loss: -4.7930


In [26]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: -5.87
initial accuracy: 0.04


In [27]:
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)

Epoch 1/10
142/238 ━━━━━━━━━━━━━━━━━━━━ 50s 523ms/step - accuracy: 0.0420 - loss: -19.6340

KeyboardInterrupt: 